<div align="right" style="text-align:right"><i>Ólafur Bjarki Bogason<br>MIT License<br>2021</i></div>

# Heimadæmi 2 leyst með Sympy

In [4]:
from sympy import symbols, Matrix, nsimplify, solve, Eq
 
def simplify(expr):
    """Simplify to a rational expression"""
    return nsimplify(expr, rational=True)

# Dæmi 1 - Finna gildi á viðnámum $R_1, R_2$

Við vitum að jafngildisviðnám fyrir tvö hliðtengd viðnám $R_1$ og $R_2$ fæst með eftirfarandi

In [104]:
def parallel(R1, R2):
    return R1*R2 / (R1+R2)

simplify(parallel(10, 2))

5/3

Sömuleiðis vitum við að $R_1 = 3 R_2$, og $R_1 \mathbin{\|} R_2 = 10k\Omega$. Nú höfum við tvær jöfnur, og tvær óþekktar sem við getum leyst með `sympy.solve`

In [105]:
def daemi_1(hlutfall=3, jafngildisvidnam=10e3):
    R1, R2 = symbols('R1 R2')

    eqn1 = Eq(R1, hlutfall*R2)
    eqn2 = Eq(parallel(R1, R2), jafngildisvidnam)

    (R1, R2), *_ = solve((eqn1, eqn2), R1, R2)

    return simplify(R1), simplify(R2)

R1_svar, R2_svar = daemi_1()

print(f'R1: {R1_svar} Ohm, R2: {R2_svar} Ohm')

R1: 15000 Ohm, R2: 7500 Ohm


# Dæmi 2 - Finnum spennur $v_a, v_b, v_c, v_d$

![Rás fyrir dæmi 2](img/hd2_2.png)

Munum að $G = 1/R$, og að við þurfum $N_{jöfnur} = N_{hnútpunktar} + N_{spennulindir} - 1 = 5 + 2 -1 = 6$. Setjum nú upp jöfnurnar og leysum með `sympy.solve`.

In [234]:
G1, G2, G3, G4, V1, V2 = {'G1': 1 / 4, 'G2': 1 / 12, 'G3': 1 / 2, 'G4': 1 / 3, 'V1': 2, 'V2': 3}.values()

va, vb, vc, vd, jv1, jv2 = symbols('va vb vc vd jv1 jv2')

kcl_a = Eq(jv1 + (va - vb) * G1, 0)
kcl_b = Eq((vb-va)*G1 + (vb-vc)*G2 + (vb-vd)*G3, 0)
kcl_c = Eq((vc-vb)*G2 + jv2, 0)
kcl_d = Eq((vd - vb)*G3 + (vd - 0)*G4, 0)

kvl_v1 = Eq(va - 0, V1)
kvl_v2 = Eq(vc - 0, V2)

va, vb, vc, vd, jv1, jv2 = solve((kcl_a, kcl_b, kcl_c, kcl_d, kvl_v1, kvl_v2), va, vb, vc, vd, jv1, jv2).values()

print(f'va: {va:.2f} V, vb: {vb:.2f} V, vc: {vc:.2f} V, vd: {vd:.2f} V')

va: 2.00 V, vb: 1.41 V, vc: 3.00 V, vd: 0.84 V


Hér er sama dæmi leyst með MNA.

In [235]:
def ras_2():
    V1, V2, G1, G2, G3, G4 = symbols('V1 V2 G1 G2 G3 G4')

    A = Matrix([
        [G1, -G1, 0, 0, 1, 0],                # KCL í a
        [-G1, G1 + G2 + G3, -G2, -G3, 0, 0],  # KCL í b
        [0, -G2, G2, 0, 0, 1],                # KCL í c
        [0, -G3, 0, G3+G4, 0, 0],             # KCL í d
        [1, 0, 0, 0, 0, 0],                   # KVL yfir V1
        [0, 0, 1, 0, 0, 0],                   # KVL yfir V2
    ])

    b = Matrix([[0, 0, 0, 0, V1, V2]]).T

    return A.LUsolve(b)

In [236]:
def daemi_2(gildi):
    va, vb, vc, vd, jv, jvcvs = ras_2().subs(gildi)

    print(f'va: {va} V, vb: {vb:.2f} V, vc: {vc} V, vd: {vd:.2f} V')

In [237]:
daemi_2(daemi_2_gildi)

va: 2 V, vb: 1.41 V, vc: 3 V, vd: 0.84 V


# Dæmi 3 - Finna $V$ ef $i_6$, straumur í grein, er gefinn

Tökum eftir því að $V = v_a - 0$

![Rás fyrir dæmi 2](img/hd2_3.png)

In [241]:
G1, G2, G3, G4, G5, G6 = {'G1': 1 / 20, 'G2': 1 / 16, 'G3': 1 / 5, 'G4': 1 / 10, 'G5': 1 / 30, 'G6': 1 / 20}.values()

V, va, vb, vc, jv = symbols('V va vb vc jv')

# Vitum spennu í vd með lögmáli Ohm
vd = 1 / G6

kcl_a = Eq(jv + (va - vb) * G1, 0)
kcl_b = Eq((vb-va)*G1 + (vb-0)*G2 + (vb-vc)*G3, 0)
kcl_c = Eq((vc-vb)*G3 + (vc - 0)*G4 + (vc - vd)*G5, 0)
kcl_d = Eq((vd - vc)*G5 + (vd - 0)*G6, 0)

kvl_v1 = Eq(va - 0, V)

V_sol, *_ = solve((kcl_a, kcl_b, kcl_c, kcl_d, kvl_v1), V, va, vb, vc, jv).values()

print(f'V: {V_sol:.2f} V')

V: 300.00 V


# Dæmi 4 - Straumlindir
![Dæmi 4](img/hd2_4.png)

In [242]:
def ras_4():
    I1, I2, I3, I4, I5 = symbols('I1 I2 I3 I4 I5')
    G1, G2, G3, G4, G5, G6, G7, G8, G9, G10 = symbols('G1 G2 G3 G4 G5 G6 G7 G8 G9 G10')

    A = Matrix([
        [G1 + G2 + G3, -G3, -G2, 0, 0],                 # KCL í a (ekki I2, er í b vigri)
        [-G3, G3 + G4 + G5, -G4, -G5, 0],               # KCL í b (ekki I1, er í b vigri)
        [-G2, -G4, G2 + G4 + G6 + G7 + G8, -G7, -G8],   # KCL í c (ekki I3, er í b vigri)
        [0, -G5, -G7, G5 + G7 + G9, -G9],               # KCL í d (ekki I5, er í b vigri)
        [0, 0, -G8, -G9, G8 + G9 + G10],                # KCL í e (ekki I4, er í b vigri)
    ])

    b = Matrix([[I2, I1, I3, I5, I4]]).T

    return A.LUsolve(b)

In [243]:
def daemi_4():
    va, vb, vc, vd, ve = ras_4().subs({
        'G1': 1, 'G2': 2, 'G3': 1, 'G4': 3, 'G5': 2, 
        'G6': 1, 'G7': 1, 'G8': 3, 'G9': 2, 'G10': 2, 
        'I1': 3, 'I2': -1, 'I3': -2, 'I4': 4, 'I5': 2
    })
    
    print(f'va: {va} V, vb: {vb} V, vc: {vc} V, vd: {vd} V, ve: {ve} V')
    print(f'va: {float(va):.4f} V, vb: {float(vb):.4f} V, vc: {float(vc):.4f} V, vd: {float(vd):.4f} V, ve: {float(ve):.4f} V')

In [244]:
daemi_4()

va: 1791/1834 V, vb: 1936/917 V, vc: 2563/1834 V, vd: 4125/1834 V, ve: 475/262 V
va: 0.9766 V, vb: 2.1112 V, vc: 1.3975 V, vd: 2.2492 V, ve: 1.8130 V


# Dæmi 4 - Ekki á heimadæmum, spennulindir í stað straumlinda
![Dæmi 4](img/hd2_4_vs.png)

In [45]:
def ras_4_spennulindir():
    V1, V2, V3, V4, V5 = symbols('V1 V2 V3 V4 V5')
    G1, G2, G3, G4, G5, G6, G7, G8, G9, G10 = symbols('G1 G2 G3 G4 G5 G6 G7 G8 G9 G10')

    A = Matrix([
        [G1 + G2 + G3, -G3, -G2, 0, 0, 0, 1, 0, 0, 0],                # KCL í a (ekki I2, er í b vigri)
        [-G3, G3 + G4 + G5, -G4, -G5, 0, 1, 0, 0, 0, 0],
        [-G2, -G4, G2 + G4 + G6 + G7 + G8, -G7, -G8, 0, 0, 1, 0, 0],
        [0, -G5, -G7, G5 + G7 + G9, -G9, 0, 0, 0, 0, 1],
        [0, 0, -G8, -G9, G8 + G9 + G10, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    ])

    b = Matrix([[0, 0, 0, 0, 0, V1, V2, V3, V4, V5]]).T

    return A.LUsolve(b)

In [46]:
def daemi_4_spennulindir():
    ras = ras_4_spennulindir().subs({
        'G1': 1, 'G2': 2, 'G3': 1, 'G4': 3, 'G5': 2,
        'G6': 1, 'G7': 1, 'G8': 3, 'G9': 2, 'G10': 2,
        'V1': 3, 'V2': -1, 'V3': -2, 'V4': 4, 'V5': 2
    })
    
    va, vb, vc, vd, ve, *_ = simplify(ras)
    
    assert (va, vb, vc, vd, ve) == (-1, 3, -2, 2, 4)
    
    print(f'va: {va} V, vb: {vb} V, vc: {vc} V, vd: {vd} V, ve: {ve} V')

In [47]:
daemi_4_spennulindir()

va: -1 V, vb: 3 V, vc: -2 V, vd: 2 V, ve: 4 V


# Dæmi 5
![Dæmi 5](img/hd2_5.png)

In [48]:
def ras_5():
    V, G1, G2, G3, G4, G5, G6 = symbols('V G1 G2 G3 G4 G5 G6')

    A = Matrix([
        [G1 + G2, -G1, -G2, 0, 1],           # KCL í a
        [-G1, G1 + G3 + G5, -G3, -G5, 0],    # KCL í b 
        [-G2, -G3, G2 + G3 + G4, -G4, 0],    # KCL í c
        [0, -G5, -G4, G4 + G5 + G6, 0],      # KCL í d
        [1, 0, 0, 0, 0],                     # Tengjum spennu yfir V1 við hnútpunkta
    ])

    b = Matrix([[0, 0, 0, 0, V]]).T          # Engin óháð straumlind svo fyrstu fjögur stökin eru núll.

    return A.LUsolve(b)

In [199]:
def daemi_5(gildi):
    va, vb, vc, vd, jv = ras_5().subs(gildi)
    i4 = (vc - vd)* gildi['G4']

    print(f'va: {va:.2f} V, vb: {vb:.2f} V, vc: {vc:.} V, vd: {vd} V, i_4: {i4} A')

In [200]:
daemi_5({'V': 1, 'G1': 1 / 1, 'G2': 1 / 1, 'G3': 1 / 1, 'G4': 1 / 1, 'G5': 1 / 1, 'G6': 1 / 1})

# Aukadæmi
daemi_5({'V': 1, 'G1': 1 / 1, 'G2': 1 / 2, 'G3': 1 / 3, 'G4': 1 / 4, 'G5': 1 / 5, 'G6': 1 / 6})

va: 1 V, vb: 0.750000000000000 V, vc: 0.750000000000000 V, vd: 0.500000000000000 V, i_4: 0.250000000000000 A
va: 1 V, vb: 0.936170212765957 V, vc: 0.904255319148936 V, vd: 0.670212765957447 V, i_4: 0.0585106382978723 A
